# Review Session 3 - Part 2: Density Regressions

Linear algebra and complexity variables

Course: Tools of Economic Complexity

**Outline**

- Linear algebra basics
- Networks - construction and metrics
	- Adjacency matrix
	- Edge lists
	- Centrality metrics
	- “Network backboning” - Michele / Frank
    - Degree distributions (exercise)
- Complexity metrics
	- ECI / PCI / Density
	- Industry spaces - co-production / co-location / co-coordination
    - Predicting product appearances
    - Backing out country CCA's
- Density regressions
	- Growth vs density
	- Product appearances vs density
- Growth regressions
    - Growth vs ECI etc. (exercise)

In [ ]:
%reset -f

In [ ]:
# Helps while coding up modules to import
%reload_ext autoreload
%autoreload 2

In [ ]:
# !pip install wbdata statsmodels ecomplexity

In [ ]:
# Basics
import os
import re
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import wbdata
from statsmodels.iolib.summary2 import summary_col

In [ ]:
# Set file paths
PROJ = Path(os.path.realpath("."))
ROOT = PROJ.parent
DATA = ROOT / "data/"

# Complexity Measures

Using minimum conditional probability method for calculating proximity
$$\phi_{pp'} = \frac{\sum_c M_{cp} M_{cp'}}{max(U_p, U_p')}$$

$$d_{cp} = \frac{\sum_c M_{cp'}\phi_{pp'}}{\sum_p \phi_{pp'}}$$

In [ ]:
# Set start year to simplify analyses
start_year = 2010

In [ ]:
# Download HS-4 trade data
data_url = f"https://intl-atlas-downloads.s3.amazonaws.com/country_hsproduct4digit_year.csv.zip"
trade = pd.read_csv(data_url, compression="zip", low_memory=False)
# Select years to include
trade = trade[trade.year >= start_year]
trade.head()

,year,export_value,import_value,export_rca,cog,distance,hs_eci,hs_coi,sitc_eci,sitc_coi,pci,location_code,location_name_short_en,hs_product_code,hs_product_name_short_en
15,2010,0.0,5350.0,0.000000,0.067729,0.935516,1.262046,-0.145284,1.156783,0.269547,-0.005620,ABW,Aruba,0101,Horses
16,2011,0.0,18966.0,0.000000,0.224593,0.988999,-0.153565,-1.006434,0.074068,-0.815275,0.433045,ABW,Aruba,0101,Horses
17,2012,0.0,29648.0,0.000000,0.080129,0.982993,0.167140,-0.967966,0.372045,-0.742569,-0.183913,ABW,Aruba,0101,Horses
18,2013,6199.0,110883.0,0.080352,0.115505,0.975545,0.487088,-0.832030,0.268743,-0.656154,0.079526,ABW,Aruba,0101,Horses
19,2014,0.0,7500.0,0.000000,0.144265,0.982488,-0.066792,-1.000541,-0.217015,-0.913747,0.270862,ABW,Aruba,0101,Horses


In [ ]:
# Proximities from the atlas
proxurl = (
    "http://intl-atlas-downloads.s3.amazonaws.com/atlas_2_16_6/hs92_proximities.csv"
)
proxdf = pd.read_csv(
    proxurl, dtype={"commoditycode_1": str, "commoditycode_2": str, "proximity": float}
)
proxdf.head()

,commoditycode_1,commoditycode_2,proximity
0,0101,0101,0.000000
1,0101,0102,0.277778
2,0101,0103,0.352941
3,0101,0104,0.260870
4,0101,0105,0.296296


In [ ]:
set(trade.hs_product_code.unique()) - set(proxdf.commoditycode_1.unique())

{'9999', 'XXXX', 'financial', 'ict', 'transport', 'travel', 'unspecified'}

In [ ]:
# Filter trade data to include "valid" products
trade = trade[trade.hs_product_code.isin(proxdf.commoditycode_1.unique())]

In [ ]:
# Rectangularize
def fillin(df, entities):
    """STATA style 'fillin', makes sure all combinations of entities in the
    index are in the dataset."""
    df = df.set_index(entities)
    df = df.reindex(pd.MultiIndex.from_product(df.index.levels, names=df.index.names))
    return df.reset_index()

In [ ]:
len(trade)

2035610

In [ ]:
# Rectangularize - fill in missing combinations
trade = fillin(trade, ["year", "location_code", "hs_product_code"])

In [ ]:
len(trade)

2057160

In [ ]:
from ecomplexity import ecomplexity, proximity

# Parameters
trade_cols = {
    "time": "year",
    "loc": "location_code",
    "prod": "hs_product_code",
    "val": "export_value",
}

# Calculate complexity
trade_complexity = ecomplexity(trade[list(trade_cols.values())], trade_cols)
trade_complexity.head()

/Users/shg309/miniconda/envs/localcid/lib/python3.8/site-packages/ecomplexity/ComplexityData.py:48: UserWarning: NaN value(s) present, coercing to zero(es)
  warnings.warn("NaN value(s) present, coercing to zero(es)")


2010
2011
2012
2013
2014
2015
2016


,location_code,hs_product_code,export_value,year,diversity,ubiquity,mcp,eci,pci,density,coi,cog,rca
0,ABW,0101,0.0,2010,77.0,21.0,0.0,1.287289,0.465101,0.065219,0.011302,0.372326,0.000000
1,ABW,0102,0.0,2010,77.0,41.0,0.0,1.287289,-0.435292,0.071094,0.011302,-0.059898,0.000000
2,ABW,0103,0.0,2010,77.0,21.0,0.0,1.287289,1.996005,0.063716,0.011302,0.571354,0.000000
3,ABW,0104,0.0,2010,77.0,34.0,0.0,1.287289,-2.108885,0.070731,0.011302,-0.292291,0.000000
4,ABW,0105,2342.0,2010,77.0,31.0,0.0,1.287289,1.034382,0.069503,0.011302,0.420561,0.039898


# Density Regressions

Similar to Hausmann, R., Stock, D. P., & Yıldırım, M. A. (2021). Implied comparative advantage. Research Policy, 104143.

Yang Li and Frank Neffke have upcoming work that further enhances our understanding of density regressions and how to do them carefully

In [ ]:
# Get WDI data
wdi = wbdata.get_dataframe(
    indicators={
        "NY.GDP.PCAP.CD": "gdp_pc",
        "SP.POP.TOTL": "population",
        "NY.GDP.MKTP.CD": "gdp",
    },
    country="all",
    data_date=(pd.to_datetime(str(start_year)), pd.to_datetime("2020")),
    convert_date=True,
).reset_index()
# Add country iso
countries = pd.DataFrame(wbdata.get_country())[["id", "name"]].rename(
    columns={"id": "location_code", "name": "country"}
)
wdi = wdi.merge(countries, on="country").rename(columns={"date": "year"})
# Convert year to int
wdi["year"] = wdi["year"].dt.year
wdi.head()

,country,year,gdp_pc,population,gdp,location_code
0,Africa Eastern and Southern,2020,1326.663658,677243299.0,8.984741e+11,AFE
1,Africa Eastern and Southern,2019,1481.425292,660046272.0,9.778092e+11,AFE
2,Africa Eastern and Southern,2018,1530.161917,643090131.0,9.840320e+11,AFE
3,Africa Eastern and Southern,2017,1536.206783,626392880.0,9.622690e+11,AFE
4,Africa Eastern and Southern,2016,1401.281053,609978946.0,8.547519e+11,AFE


In [ ]:
# Pre-process population for RPOP
pop_df = fillin(wdi[["year", "location_code", "population"]], ["year", "location_code"])
pop_df.head()

,year,location_code,population
0,2010,ABW,101665.0
1,2010,AFE,518468229.0
2,2010,AFG,29185511.0
3,2010,AFW,350556886.0
4,2010,AGO,23356247.0


In [ ]:
# Calculate rpop or RpCA
trade_complexity_rpop = ecomplexity(
    trade.loc[trade.year == trade.year.max(), list(trade_cols.values())],
    trade_cols,
    presence_test="rpop",
    rpop_mcp_threshold=1,
    pop=pop_df,
    verbose=True,
)
trade_complexity_rpop.head()

/Users/shg309/miniconda/envs/localcid/lib/python3.8/site-packages/ecomplexity/ComplexityData.py:48: UserWarning: NaN value(s) present, coercing to zero(es)
  warnings.warn("NaN value(s) present, coercing to zero(es)")


2016


/Users/shg309/miniconda/envs/localcid/lib/python3.8/site-packages/ecomplexity/ecomplexity.py:61: UserWarning: In year 2016, diversity / ubiquity is 0 for some locs/prods
  warnings.warn(


,location_code,hs_product_code,export_value,year,diversity,ubiquity,mcp,eci,pci,density,coi,cog,rca,rpop
0,ABW,0101,0.0,2016,39.0,36.0,0.0,NaN,NaN,0.035095,0.349719,NaN,0.0,0.0
1,ABW,0102,0.0,2016,39.0,37.0,0.0,NaN,NaN,0.032941,0.349719,NaN,0.0,0.0
2,ABW,0103,0.0,2016,39.0,22.0,0.0,NaN,NaN,0.030211,0.349719,NaN,0.0,0.0
3,ABW,0104,0.0,2016,39.0,28.0,0.0,NaN,NaN,0.034100,0.349719,NaN,0.0,0.0
4,ABW,0105,0.0,2016,39.0,30.0,0.0,NaN,NaN,0.031795,0.349719,NaN,0.0,0.0


In [ ]:
# Select a base year and an end year
base_year = 2010
end_year = 2016

In [ ]:
# Custom proximities if needed
prod_prox = proximity(
    trade.loc[trade.year == base_year, list(trade_cols.values())],
    cols_input=trade_cols,
)
country_prox = proximity(
    trade.loc[trade.year == base_year, list(trade_cols.values())],
    cols_input={
        "time": "year",
        "loc": "hs_product_code",
        "prod": "location_code",
        "val": "export_value",
    },
)

2010
2010


In [ ]:
# Get densities directly
get_density = lambda df, cols_input: ecomplexity(
    df.loc[df.year == base_year, list(cols_input.values())], cols_input=cols_input
)[list(cols_input.values()) + ["density", "mcp", "rca"]]

prod_density = get_density(trade, trade_cols)
country_density = get_density(
    trade,
    {
        "time": "year",
        "loc": "hs_product_code",
        "prod": "location_code",
        "val": "export_value",
    },
)

/Users/shg309/miniconda/envs/localcid/lib/python3.8/site-packages/ecomplexity/ComplexityData.py:48: UserWarning: NaN value(s) present, coercing to zero(es)
  warnings.warn("NaN value(s) present, coercing to zero(es)")


2010
2010


In [ ]:
location_mean_rca = (
    prod_density.groupby(["year", "location_code"])["rca"].mean().reset_index()
).rename(columns={"rca": "location_mean_rca"})
location_mean_rca.head()

,year,location_code,location_mean_rca
0,2010,ABW,0.506751
1,2010,AFG,4.025953
2,2010,AGO,0.030467
3,2010,AIA,1.654647
4,2010,ALB,1.473813


In [ ]:
# Merge densities
id_cols = ["year", "hs_product_code", "location_code"]
density_df = (
    prod_density.rename(columns={"density": "prod_space_density"})[
        id_cols + ["prod_space_density", "rca", "mcp"]
    ]
    .merge(
        country_density.rename(columns={"density": "country_space_density"})[
            id_cols + ["country_space_density"]
        ],
        on=id_cols,
    )
    .merge(location_mean_rca, on=["year", "location_code"], how="inner")
)
density_df.head()

,year,hs_product_code,location_code,prod_space_density,rca,mcp,country_space_density,location_mean_rca
0,2010,0101,ABW,0.065219,0.000000,0.0,0.100076,0.506751
1,2010,0102,ABW,0.071094,0.000000,0.0,0.190783,0.506751
2,2010,0103,ABW,0.063716,0.000000,0.0,0.100361,0.506751
3,2010,0104,ABW,0.070731,0.000000,0.0,0.159486,0.506751
4,2010,0105,ABW,0.069503,0.039898,0.0,0.145571,0.506751


In [ ]:
# Compute growth rates
exports = trade.loc[
    trade.year.isin([base_year, end_year]),
    ["year", "location_code", "hs_product_code", "export_value"],
]
exports = (
    exports.pivot(
        index=["location_code", "hs_product_code"],
        columns="year",
        values="export_value",
    )
    .rename_axis(None, axis=1)
    .reset_index()
)
# Intensive margin only, also avoid -infs
exports = exports[(exports[base_year] > 0) & (exports[end_year] > 0)]
# Get annualized growth rates
exports["growth_rate"] = np.log(exports[end_year] / exports[base_year]) / (
    end_year - base_year
)
product_sums = exports.groupby("hs_product_code")[[base_year, end_year]].transform(
    "sum"
)
location_sums = exports.groupby("location_code")[[base_year, end_year]].transform("sum")
exports["radial_product_growth"] = np.log(
    product_sums[end_year] / product_sums[base_year]
) / (end_year - base_year)
exports["radial_location_growth"] = np.log(
    location_sums[end_year] / location_sums[base_year]
) / (end_year - base_year)
exports["base_year_location_total"] = location_sums[base_year]
exports["base_year_product_total"] = product_sums[base_year]
exports["base_year_exports"] = exports[base_year]
exports = exports.drop(columns=[base_year, end_year])
exports.head()

,location_code,hs_product_code,growth_rate,radial_product_growth,radial_location_growth,base_year_location_total,base_year_product_total,base_year_exports
5,ABW,0106,0.485847,-0.000913,-0.129283,303195493.0,1.086489e+09,171.0
137,ABW,1515,-0.307337,0.041754,-0.129283,303195493.0,3.110871e+09,49096.0
153,ABW,1704,0.196791,0.039206,-0.129283,303195493.0,8.393139e+09,331.0
159,ABW,1806,0.445507,0.048578,-0.129283,303195493.0,1.947883e+10,24717.0
163,ABW,1904,-0.818201,0.028372,-0.129283,303195493.0,5.007930e+09,15044.0


In [ ]:
# Put everything together
reg_df = density_df.merge(
    exports, on=["location_code", "hs_product_code"], how="inner"
).merge(wdi[["year", "location_code", "population"]], on=["year", "location_code"])
log_vars = [
    "rca",
    "growth_rate",
    "base_year_exports",
    "population",
    "base_year_location_total",
    "base_year_product_total",
    "prod_space_density",
    "country_space_density",
]
for x in log_vars:
    reg_df[f"{x}_log"] = np.log(reg_df[f"{x}"])
# No infs
reg_df = reg_df[~reg_df.growth_rate_log.isin([np.inf, -np.inf])]
reg_df.head()

/Users/shg309/miniconda/envs/localcid/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/shg309/miniconda/envs/localcid/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,year,hs_product_code,location_code,prod_space_density,rca,mcp,country_space_density,location_mean_rca,growth_rate,radial_product_growth,...,base_year_exports,population,rca_log,growth_rate_log,base_year_exports_log,population_log,base_year_location_total_log,base_year_product_total_log,prod_space_density_log,country_space_density_log
0,2010,0106,ABW,0.071017,0.006122,0.0,0.281551,0.506751,0.485847,-0.000913,...,171.0,101665.0,-5.095822,-0.721862,5.141664,11.529438,19.529888,20.806217,-2.644834,-1.267440
1,2010,1515,ABW,0.064852,0.642764,0.0,0.160443,0.506751,-0.307337,0.041754,...,49096.0,101665.0,-0.441978,NaN,10.801533,11.529438,19.529888,21.858169,-2.735654,-1.829818
2,2010,1704,ABW,0.070934,0.001607,0.0,0.308086,0.506751,0.196791,0.039206,...,331.0,101665.0,-6.433617,-1.625614,5.802118,11.529438,19.529888,22.850680,-2.646003,-1.177377
3,2010,1806,ABW,0.071465,0.051694,0.0,0.232894,0.506751,0.445507,0.048578,...,24717.0,101665.0,-2.962412,-0.808541,10.115247,11.529438,19.529888,23.692594,-2.638551,-1.457171
4,2010,1904,ABW,0.072733,0.122383,0.0,0.179793,0.506751,-0.818201,0.028372,...,15044.0,101665.0,-2.100597,NaN,9.618735,11.529438,19.529888,22.334289,-2.620962,-1.715951


In [ ]:
# First stage regressions
reg1_model = smf.ols(
    formula="rca_log ~ prod_space_density_log", data=reg_df, missing="drop"
).fit()
reg2_model = smf.ols(
    formula="rca_log ~ country_space_density_log", data=reg_df, missing="drop"
).fit()
reg3_model = smf.ols(
    formula="rca_log ~ prod_space_density_log + country_space_density_log",
    data=reg_df,
    missing="drop",
).fit()

In [ ]:
print(
    summary_col(
        [reg1_model, reg2_model, reg3_model],
        stars=True,
        float_format="%0.3f",
        regressor_order=[
            "Intercept",
            "prod_space_density_log",
            "country_space_density_log",
        ],
        drop_omitted=True,
        model_names=["(1)", "(2)", "(3)"],
        info_dict={
            "N": lambda x: "{0:d}".format(int(x.nobs)),
        },
    )
)


                            (1)      (2)      (3)   
----------------------------------------------------
Intercept                 0.904*** 2.576*** 2.761***
                          (0.014)  (0.019)  (0.019) 
prod_space_density_log    1.517***          0.863***
                          (0.006)           (0.007) 
country_space_density_log          2.414*** 1.614***
                                   (0.009)  (0.011) 
R-squared                 0.295    0.326    0.386   
R-squared Adj.            0.295    0.326    0.386   
N                         136829   136829   136829  
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [ ]:
# Get residuals
reg_df["residual_hybrid_density"] = reg3_model.resid
reg_df["residual_prod_space_density"] = reg1_model.resid
reg_df["residual_country_space_density"] = reg2_model.resid

In [ ]:
# Second stage
reg1_model = smf.ols(
    formula="growth_rate_log ~ residual_prod_space_density", data=reg_df, missing="drop"
).fit()
reg2_model = smf.ols(
    formula="growth_rate_log ~ residual_country_space_density",
    data=reg_df,
    missing="drop",
).fit()
reg3_model = smf.ols(
    formula="growth_rate_log ~ residual_hybrid_density", data=reg_df, missing="drop"
).fit()
reg4_model = smf.ols(
    formula="growth_rate_log ~ residual_hybrid_density + base_year_exports_log + population_log +"
    "base_year_product_total_log + location_mean_rca",
    data=reg_df,
    missing="drop",
).fit()
reg5_model = smf.ols(
    formula="growth_rate_log ~ base_year_exports_log + population_log +"
    "base_year_product_total_log + location_mean_rca + radial_product_growth + radial_location_growth",
    data=reg_df,
    missing="drop",
).fit()
reg6_model = smf.ols(
    formula="growth_rate_log ~ residual_hybrid_density + base_year_exports_log + population_log +"
    "base_year_product_total_log + location_mean_rca + radial_product_growth + radial_location_growth",
    data=reg_df,
    missing="drop",
).fit()
# reg7_model = smf.ols(
#     formula="growth_rate_log ~ residual_hybrid_density + base_year_exports_log + C(hs_product_code) + C(location_code)",
#     data=reg_df,
#     missing="drop",
# ).fit()

In [ ]:
modlist = [reg1_model, reg2_model, reg3_model, reg4_model, reg5_model, reg6_model]
print(
    summary_col(
        modlist,
        stars=True,
        float_format="%0.3f",
        regressor_order=[
            "Intercept",
            "residual_prod_space_density",
            "residual_country_space_density",
            "residual_hybrid_density",
            "base_year_exports_log",
            "population_log",
            "base_year_product_total_log",
            "location_mean_rca",
            "radial_product_growth",
            "radial_location_growth",
        ],
        drop_omitted=True,
        model_names=[f"({x})" for x in range(1, len(modlist) + 1)],
        info_dict={
            "N": lambda x: "{0:d}".format(int(x.nobs)),
        },
    )
)


                                  (1)       (2)       (3)       (4)       (5)       (6)   
------------------------------------------------------------------------------------------
Intercept                      -2.367*** -2.372*** -2.376*** -2.020*** -2.523*** -2.085***
                               (0.005)   (0.005)   (0.005)   (0.079)   (0.075)   (0.079)  
residual_prod_space_density    -0.215***                                                  
                               (0.002)                                                    
residual_country_space_density           -0.234***                                        
                                         (0.002)                                          
residual_hybrid_density                            -0.231*** -0.050***           -0.050***
                                                   (0.002)   (0.003)             (0.003)  
base_year_exports_log                                        -0.166*** -0.193*** -0.170**